In [1]:
import pandas as pd
import numpy as np
from datetime import datetime
import seaborn as sns; sns.set()
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

In [2]:
# get the 4th root for embedding: https://developers.googleblog.com/2017/11/introducing-tensorflow-feature-columns.html

In [3]:
# the combined forex and economic news data
df = pd.read_csv('fx_with_news.csv', header=[0,1], index_col=0)

In [4]:
df

c_eur                                                     \
                      mean       std bar_len bar_spearman bar_log_r   first_r   
ctime                                                                           
2016-01-03 22:00:00  1.088  0.000015    28.0     -0.73730 -0.000037  0.000030   
2016-01-03 22:05:00  1.087  0.000049   155.0     -0.18100 -0.000230  0.000081   
2016-01-03 22:10:00  1.087  0.000105   138.0     -0.46040 -0.000248  0.000202   
2016-01-03 22:15:00  1.087  0.000043   125.0      0.28220 -0.000028 -0.000007   
2016-01-03 22:20:00  1.087  0.000148   115.0     -0.04312 -0.000120 -0.000144   
...                    ...       ...     ...          ...       ...       ...   
2019-12-31 21:35:00  1.121  0.000036    62.0     -0.80100 -0.000080  0.000062   
2019-12-31 21:40:00  1.121  0.000028    41.0     -0.14160 -0.000009 -0.000027   
2019-12-31 21:45:00  1.121  0.000046   108.0      0.61300 -0.000018  0.000029   
2019-12-31 21:50:00  1.121  0.000046    97.0     -0.91260 -0.000143  0.000056   
2019-12-31 21:55:00  1.121  0.000131    91.0      0.07510  0.000294 -0.000033   

                                                                     ...  \
                        max_r     min_r    last_r bar_quantile_25_r  ...   
ctime                                                                ...   
2016-01-03 22:00:00  0.000030 -0.000016 -0.000007         -0.000009  ...   
2016-01-03 22:05:00  0.000081 -0.000149 -0.000149         -0.000029  ...   
2016-01-03 22:10:00  0.000202 -0.000147 -0.000046         -0.000064  ...   
2016-01-03 22:15:00  0.000057 -0.000108 -0.000035         -0.000007  ...   
2016-01-03 22:20:00  0.000224 -0.000264 -0.000264         -0.000126  ...   
...                       ...       ...       ...               ...  ...   
2019-12-31 21:35:00  0.000062 -0.000036 -0.000018         -0.000027  ...   
2019-12-31 21:40:00  0.000035 -0.000054 -0.000036         -0.000018  ...   
2019-12-31 21:45:00  0.000047 -0.000096  0.000011         -0.000024  ...   
2019-12-31 21:50:00  0.000092 -0.000122 -0.000086         -0.000015  ...   
2019-12-31 21:55:00  0.000261 -0.000122  0.000261         -0.000078  ...   

                                                             after_counter_ohe  \
                    _united states nondefense capital goods orders ex aircraft   
ctime                                                                            
2016-01-03 22:00:00                                           0.000000           
2016-01-03 22:05:00                                           0.000000           
2016-01-03 22:10:00                                           0.000000           
2016-01-03 22:15:00                                           0.000000           
2016-01-03 22:20:00                                           0.000000           
...                                                                ...           
2019-12-31 21:35:00                                           0.429514           
2019-12-31 21:40:00                                           0.429167           
2019-12-31 21:45:00                                           0.428819           
2019-12-31 21:50:00                                           0.428472           
2019-12-31 21:55:00                                           0.428125           

                                                     \
                    _united states nonfarm payrolls   
ctime                                                 
2016-01-03 22:00:00                             0.0   
2016-01-03 22:05:00                             0.0   
2016-01-03 22:10:00                             0.0   
2016-01-03 22:15:00                             0.0   
2016-01-03 22:20:00                             0.0   
...                                             ...   
2019-12-31 21:35:00                             0.0   
2019-12-31 21:40:00                             0.0   
2019-12-31 21:45:00                             0.0   
2019-12-31 2

In [5]:
# column hierarchy
for top in list(df.columns.get_level_values(0).unique()):
    print('----------------------------------')
    print('TOP COLUMN: ', top)
    print('SUBCOLUMNS: ', len(df[top].columns))
    print(list(df[top].columns))


----------------------------------
TOP COLUMN:  c_eur
SUBCOLUMNS:  11
['mean', 'std', 'bar_len', 'bar_spearman', 'bar_log_r', 'first_r', 'max_r', 'min_r', 'last_r', 'bar_quantile_25_r', 'bar_quantile_75_r']
----------------------------------
TOP COLUMN:  c_gbp
SUBCOLUMNS:  11
['mean', 'std', 'bar_len', 'bar_spearman', 'bar_log_r', 'first_r', 'max_r', 'min_r', 'last_r', 'bar_quantile_25_r', 'bar_quantile_75_r']
----------------------------------
TOP COLUMN:  c_jpy
SUBCOLUMNS:  11
['mean', 'std', 'bar_len', 'bar_spearman', 'bar_log_r', 'first_r', 'max_r', 'min_r', 'last_r', 'bar_quantile_25_r', 'bar_quantile_75_r']
----------------------------------
TOP COLUMN:  month
SUBCOLUMNS:  12
['_1', '_2', '_3', '_4', '_5', '_6', '_7', '_8', '_9', '_10', '_11', '_12']
----------------------------------
TOP COLUMN:  dow
SUBCOLUMNS:  6
['_0', '_1', '_2', '_3', '_4', '_6']
----------------------------------
TOP COLUMN:  hour
SUBCOLUMNS:  24
['_0', '_1', '_2', '_3', '_4', '_5', '_6', '_7', '_8', '_9',

In [6]:
top_level = list(df.columns.get_level_values(0).unique())
top_level

['c_eur',
 'c_gbp',
 'c_jpy',
 'month',
 'dow',
 'hour',
 'event_cur',
 'event_exist',
 'actual_ohe',
 'surprise_ohe',
 'change_ohe',
 'after_counter_ohe']

In [7]:
# we will use 2016-2018 as training data, and 2019 as validation and test data
# problem: we have events that occure only in 2019
# we can't normalize/standardize/scale them based on 2016-2018 data
# we could move them to the '_(currency) event' columns, 
# but there is only a few, so here I will delete them

# have we all event type in 2016-2018 yers?
df['event_exist'].loc['2016':'2019'].any().all()

False

In [8]:
# find events occuring only in 2019
event_before_19 = df['event_exist'].loc['2016':'2019'].any()
event_only_19 = list(event_before_19[event_before_19==False].index)
event_only_19

['_european monetary union markit pmi composite',
 '_germany zew survey  economic sentiment',
 '_japan gross domestic product qoq',
 '_united states consumer price index ex food  energy mom',
 '_united states initial jobless claims 4week average',
 '_united states nondefense capital goods orders ex aircraft']

In [9]:
# drop events only occuring in 2019
df.drop(event_only_19, axis=1, level=1, inplace=True)

In [10]:
# have we all event type in 2016-2018 yers?
df['event_exist'].loc['2016':'2019'].any().all()
#ok

True

In [11]:
# label is unstandardized
# mean of a bar is less noisy than open or close, probably easier to recognize patterns
df_label = df.xs('mean', axis=1, level=1, drop_level=False)
df_label

,c_eur,c_gbp,c_jpy
,mean,mean,mean
ctime,,,
2016-01-03 22:00:00,1.088,1.474,0.8315
2016-01-03 22:05:00,1.087,1.475,0.8315
2016-01-03 22:10:00,1.087,1.475,0.8315
2016-01-03 22:15:00,1.087,1.475,0.8315
2016-01-03 22:20:00,1.087,1.475,0.8315
...,...,...,...
2019-12-31 21:35:00,1.121,1.325,0.9204
2019-12-31 21:40:00,1.121,1.325,0.9204


In [12]:
# we use only train and validation
# Standardizing and normalizing shifts the dataset, where +- sign can be important. Later the model will adjust this by the biases,
# but the information is lost. It could improve the performance if we make a new feature for + and - numbers, but here I don't want to make more features

In [13]:
c_list = ['c_eur', 'c_gbp', 'c_jpy']
# spearman doesn't need standardization
c_single_features = ['mean', 'std', 'bar_len']

# standardize features based on data up to 2019-01-01 00:00
for cur in c_list:
    for feature in c_single_features:
        df.loc[:,(cur, feature)] = (df[cur, feature] - df[cur, feature].loc['2016':'2019'].mean())/df[cur, feature].loc['2016':'2019'].std()

In [14]:
c_related_features = ['bar_log_r','first_r','max_r','min_r','last_r','bar_quantile_25_r','bar_quantile_75_r']
# these are related feature, we can think of them as one feature with subfeatures
# they are on similar scale 
# and the pairs on the other side of the mean (like max_r and min_r) are almost symmetric
# to preserve the signs we don't subtract the mean, only divide by the std
# this way we preserve their relative size
# *10000000 is only for easier comparison
print(df['c_eur'][c_related_features].abs().mean()*10000000)
print(df['c_eur'][c_related_features].mean()*10000000)

bar_log_r            1756.304442
first_r              1079.158825
max_r                1680.076339
min_r                1686.788351
last_r                960.659008
bar_quantile_25_r     647.481262
bar_quantile_75_r     647.172419
dtype: float64
bar_log_r               8.381825
first_r                -0.841197
max_r                1680.076339
min_r               -1686.788351
last_r                  7.541522
bar_quantile_25_r    -647.041006
bar_quantile_75_r     646.659303
dtype: float64


In [15]:
related_std = df[cur, 'bar_log_r'].loc['2016':'2019'].std()

for cur in c_list:
    for feature in c_related_features:
        df.loc[:,(cur, feature)] = (df[cur, feature])/related_std

In [16]:
# ok, these features remained on similar scale, but the other features as well
print(df['c_eur'][c_related_features].abs().mean())
print(df['c_eur'][c_related_features].mean())

bar_log_r            0.493106
first_r              0.302988
max_r                0.471704
min_r                0.473588
last_r               0.269718
bar_quantile_25_r    0.181789
bar_quantile_75_r    0.181702
dtype: float64
bar_log_r            0.002353
first_r             -0.000236
max_r                0.471704
min_r               -0.473588
last_r               0.002117
bar_quantile_25_r   -0.181665
bar_quantile_75_r    0.181558
dtype: float64


In [17]:
# 'event_cur' is on similar scale, we don't touch (normalization would be better than standardization)
df['event_cur'].max()

_EUR    2.000000
_GBP    2.828427
_JPY    1.732051
_USD    3.000000
dtype: float64

In [18]:
# should consider every event type alone, but there are 76 event types, so just normalize
df['actual_ohe'] = (df['actual_ohe'] - df['actual_ohe'].loc['2016':'2019'].min())/(df['actual_ohe'].loc['2016':'2019'].max() - df['actual_ohe'].loc['2016':'2019'].min())
# where max-min == 0 we get NaN, fill with 0
df['actual_ohe'] = df['actual_ohe'].fillna(0.0)

In [19]:
# dividing with the max of absolute values keeps the signs
df['surprise_ohe'] = df['surprise_ohe']/df['surprise_ohe'].loc['2016':'2019'].abs().max()
df['surprise_ohe'] = df['surprise_ohe'].fillna(0.0)

df['change_ohe'] = df['change_ohe']/df['change_ohe'].loc['2016':'2019'].abs().max()
df['change_ohe'] = df['change_ohe'].fillna(0.0)

df['after_counter_ohe'] = df['after_counter_ohe']/df['after_counter_ohe'].loc['2016':'2019'].abs().max()
df['after_counter_ohe'] = df['after_counter_ohe'].fillna(0.0)


In [20]:
df_label

,c_eur,c_gbp,c_jpy
,mean,mean,mean
ctime,,,
2016-01-03 22:00:00,1.088,1.474,0.8315
2016-01-03 22:05:00,1.087,1.475,0.8315
2016-01-03 22:10:00,1.087,1.475,0.8315
2016-01-03 22:15:00,1.087,1.475,0.8315
2016-01-03 22:20:00,1.087,1.475,0.8315
...,...,...,...
2019-12-31 21:35:00,1.121,1.325,0.9204
2019-12-31 21:40:00,1.121,1.325,0.9204


In [97]:
target_size=4
train_size=4
shift=1
batch_size=2

In [98]:
# label data begins from the end of the first train data
def make_label_ds(target_size, train_size, shift, df_label):
    ds_label = tf.data.Dataset.from_tensor_slices(df_label.values[train_length:])
    ds_label = ds_label.window(target_size, shift=shift, drop_remainder=True)
    return ds_label.flat_map(lambda window: window.batch(target_size))

In [117]:
ds_label = make_label_ds(target_size, train_size, shift, df_label)
next(iter(ds_label.batch(batch_size)))

<tf.Tensor: shape=(2, 4, 3), dtype=float64, numpy=
array([[[1.087, 1.474, 0.831],
        [1.087, 1.474, 0.831],
        [1.086, 1.474, 0.831],
        [1.086, 1.474, 0.831]],

       [[1.087, 1.474, 0.831],
        [1.086, 1.474, 0.831],
        [1.086, 1.474, 0.831],
        [1.086, 1.474, 0.831]]])>

In [118]:
def make_input_ds(train_size, shift, df_input):
    ds_input = tf.data.Dataset.from_tensor_slices(df_input.values)
    ds_input = ds_input.window(train_size, shift=shift, drop_remainder=True)
    return ds_input.flat_map(lambda window: window.batch(train_size))

In [125]:
# ['c_eur', 'c_gbp', 'c_jpy', 'month', 'dow', 'hour', 'event_cur', 'event_exist', 'actual_ohe', 'surprise_ohe', 'change_ohe', 'after_counter_ohe']
ds_eur = make_label_ds(target_size, train_size, shift, df['c_eur'])
ds_fbp = make_label_ds(target_size, train_size, shift, df['c_gbp'])
ds_jpy = make_label_ds(target_size, train_size, shift, df['c_jpy'])
ds_time = make_label_ds(target_size, train_size, shift, df[['month', 'dow', 'hour']])
ds_event_cur = make_label_ds(target_size, train_size, shift, df['event_cur'])
ds_event_exist = make_label_ds(target_size, train_size, shift, df['event_exist'])
ds_actual_ohe = make_label_ds(target_size, train_size, shift, df['actual_ohe'])
ds_surprise_ohe = make_label_ds(target_size, train_size, shift, df['surprise_ohe'])
ds_change_ohe = make_label_ds(target_size, train_size, shift, df['change_ohe'])
ds_after_counter_ohe = make_label_ds(target_size, train_size, shift, df['after_counter_ohe'])

In [126]:
next(iter(ds_change_ohe.batch(batch_size)))

<tf.Tensor: shape=(2, 4, 70), dtype=float64, numpy=
array([[[0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
         0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
    

In [127]:
df['change_ohe']

,_eur event,_eur speech,_european monetary union consumer price index core yoy,_european monetary union consumer price index yoy,_european monetary union ecb deposit rate decision,_european monetary union ecb interest rate decision,_european monetary union gross domestic product sa qoq,_european monetary union gross domestic product sa yoy,_france markit manufacturing pmi,_gbp event,...,_united states new home sales mom,_united states nonfarm payrolls,_united states retail sales control group,_united states retail sales ex autos mom,_united states retail sales mom,_united states reutersmichigan consumer sentiment index,_united states trade balance,_united states unemployment rate,_usd event,_usd speech
ctime,,,,,,,,,,,,,,,,,,,,,
2016-01-03 22:00:00,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.0
2016-01-03 22:05:00,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.0
2016-01-03 22:10:00,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.0
2016-01-03 22:15:00,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.0
2016-01-03 22:20:00,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.000000,0.00,0.000000,0.000000,0.000000,0.00000,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2019-12-31 21:35:00,0.0,0.0,0.5,0.428522,0.0,0.0,0.0,0.0,-0.608659,0.0,...,0.090941,0.398551,-0.25,-0.181838,-0.117605,0.646755,-0.44816,0.0,0.0,0.0
2019-12-31 21:40:00,0.0,0.0,0.5,0.428522,0.0,0.0,0.0,0.0,-0.608659,0.0,...,0.090941,0.398551,-0.25,-0.181838,-0.117605,0.646755,-0.44816,0.0,0.0,0.0
2019-12-31 21:45:00,0.0,0.0,0.5,0.428522,0.0,0.0,0.0,0.0,-0.608659,0.0,...,0.090941,0.398551,-0.25,-0.181838,-0.117605,0.646755,-0.44816,0.0,0.0,0.0


In [12]:
class GatedActivationUnit(keras.layers.Layer):
    def __init__(self, activation="tanh", **kwargs):
        super().__init__(**kwargs)
        self.activation = keras.activations.get(activation)
    def call(self, inputs):
        n_filters = inputs.shape[-1] // 2
        linear_output = self.activation(inputs[..., :n_filters])
        gate = keras.activations.sigmoid(inputs[..., n_filters:])
        return self.activation(linear_output) * gate

In [13]:
def wavenet_residual_block(inputs, n_filters, dilation_rate):
    z = keras.layers.Conv1D(2 * n_filters, kernel_size=2, padding="causal",
                            dilation_rate=dilation_rate)(inputs)
    z = GatedActivationUnit()(z)
    z = keras.layers.Conv1D(n_filters, kernel_size=1)(z)
    return keras.layers.Add()([z, inputs]), z

In [23]:

keras.backend.clear_session()
np.random.seed(42)
tf.random.set_seed(42)

n_layers_per_block = 3 # 10 in the paper
n_blocks = 2 # 3 in the paper
n_filters = 32 # 128 in the paper
n_outputs = 50 # 256 in the paper

inputs = keras.layers.Input(shape=[None, 11])
z = keras.layers.Conv1D(n_filters, kernel_size=2, padding="causal")(inputs)
skip_to_last = []
for dilation_rate in [2**i for i in range(n_layers_per_block)] * n_blocks:
    z, skip = wavenet_residual_block(z, n_filters, dilation_rate)
    skip_to_last.append(skip)
z = keras.activations.relu(keras.layers.Add()(skip_to_last))
z = keras.layers.Conv1D(n_filters, kernel_size=1, activation="relu")(z)
Y_proba = keras.layers.Conv1D(n_outputs, kernel_size=1)(z)

model_wn = keras.models.Model(inputs=[inputs], outputs=[Y_proba])

In [24]:
#keras.utils.plot_model(model, show_shapes=True)
model_wn.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, 11)]   0                                            
__________________________________________________________________________________________________
conv1d (Conv1D)                 (None, None, 32)     736         input_1[0][0]                    
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, None, 64)     4160        conv1d[0][0]                     
__________________________________________________________________________________________________
gated_activation_unit (GatedAct (None, None, 32)     0           conv1d_1[0][0]                   
______________________________________________________________________________________________